In [1]:
from optionalyzer.blackscholes import BlackScholes
import pandas as pd
import datetime
import numpy as np
import yfinance

In [2]:
tesla = yfinance.Ticker("TSLA")

#Load at the money options
options = tesla.option_chain(date="2023-04-21")
calls = options.calls
puts = options.puts

spot_price = tesla.history(period="1d")['Close'][0]
print(spot_price)

#At the money options
calls = calls[((calls['strike'] - spot_price) < 10) & ((calls['strike'] - spot_price) > -10)]
puts = puts[((puts['strike'] - spot_price) < 10) & ((puts['strike'] - spot_price) > -10)]


113.05999755859375


In [3]:
calls = calls[["lastTradeDate", "strike", "lastPrice", "percentChange"]]
puts = puts[["lastTradeDate", "strike", "lastPrice", "percentChange"]]

In [4]:
calls

,lastTradeDate,strike,lastPrice,percentChange
11,2023-01-06 20:59:04+00:00,105.00,22.80,8.830542
12,2023-01-06 19:44:25+00:00,108.33,21.50,11.979162
13,2023-01-06 20:54:40+00:00,110.00,20.10,10.136989
14,2023-01-06 20:48:27+00:00,115.00,17.45,9.473032
15,2023-01-06 19:13:27+00:00,116.67,16.70,10.231030
16,2023-01-06 20:56:07+00:00,120.00,15.45,11.471863


In [5]:
puts

,lastTradeDate,strike,lastPrice,percentChange
11,2023-01-06 20:52:15+00:00,105.00,13.20,-6.382982
12,2023-01-06 20:59:25+00:00,108.33,14.75,-4.220777
13,2023-01-06 20:56:54+00:00,110.00,15.52,-5.939391
14,2023-01-06 20:54:55+00:00,115.00,18.00,-5.013197
15,2023-01-06 20:44:24+00:00,116.67,18.97,-4.912288
16,2023-01-06 20:57:55+00:00,120.00,20.74,-4.993130


In [6]:
strikes = calls['strike'].values
call_prices = calls['lastPrice'].values

put_prices = puts['lastPrice'].values


In [7]:
bs = BlackScholes()

In [8]:
df = pd.read_csv("../../Data/Stocks/nasdaq200/TSLA.csv", parse_dates=["Date"], index_col="Date")
df = df[df.index>=datetime.datetime(2016,1,1)]
return_ = df["Close"].pct_change(1).dropna()
log_return = np.log(1+return_)
sigma = log_return.std()*np.sqrt(252)
print(sigma)

0.5854294125474527


In [9]:
today = datetime.datetime.today()
april_expiry_date = datetime.datetime(today.year, 4, 21)

num_days = (april_expiry_date - today).days
print(num_days)

102


In [10]:
S = spot_price
id_ = 1
K = strikes[id_]
r = 0.0342
tau = num_days/252

call_price = bs.call(S,K,r,sigma, tau)
put_price = bs.put(S,K,r,sigma, tau)

print("Calculated Values:", call_price, put_price)
print("Trading Values:", call_prices[id_], put_prices[id_])

Calculated Values: 19.537864858888383 13.31860220954642
Trading Values: 21.5 14.75


In [11]:
S = spot_price
id_ = 4
K = strikes[id_]
r = 0.0342
tau = num_days/252

call_price = bs.call(S,K,r,sigma, tau)
put_price = bs.put(S,K,r,sigma, tau)

print("Calculated Values:", call_price, put_price)
print("Trading Values:", call_prices[id_], put_prices[id_])

Calculated Values: 15.866494878394235 17.872578201019053
Trading Values: 16.7 18.97


In [12]:
call_volas = []
put_volas = []

for i in range(len(strikes)):
    call_vola =  bs.implied_volatility(call_prices[i], S=S, K=strikes[i], r=r, tau=tau, verbose=0, option_type="call")
    put_vola =  bs.implied_volatility(put_prices[i], S=S, K=strikes[i], r=r, tau=tau, verbose=0, option_type="put")

    call_volas.append(call_vola)
    put_volas.append(put_vola)

In [13]:
call_volas

[0.646726641414932,
 0.6578991288547522,
 0.6346324270076054,
 0.6173089636017863,
 0.6147819651193667,
 0.6159793391940399]

In [14]:
put_volas

[0.6430543993629403,
 0.638280587853104,
 0.6343485117795314,
 0.6240839897173708,
 0.6240864560841277,
 0.6159687051390069]